<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=fa1b89b34f41912c6047be2563f8cb0051d2387628ec23cc426f4cb753401cfa
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-04 13:03:10
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.44 C
-------------------
Today PnL: 12.57 K (0.09%)
Current PnL: -24.39 L (-15.75%)
CY Booked + Current PnL: -10.10 L (-6.52%)
-------------------
Total profit:  1.30 L
Total loss:  -25.69 L
-------------------
Total Booked + Current PnL: 16.88 L (13.21%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (9.93%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 89.02 L (61.87%)
Deployed:  1.28 C
Current:  1.44 C
CAGR/XIRR %: 6.74%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.12,-7.65,8.31,0.02,13396.0,-13359.0,161200.0,146.09,68.0,M-SC,15.57,234.0,-1.00,1.16,40.22,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.03,-16.68,20.13,0.09,16902.0,-16813.0,83964.0,94.88,37.0,M-SC,2.75,253.0,-0.99,0.60,10.31,OX40N,NTT,DURABLES
43,ITC,452.00,0.45,-1.71,12.35,10.44,24296.0,-3413.0,196725.0,-41.15,44.0,X-LC,1.58,5.0,-0.14,1.42,3.76,X40,NTT,FMCG
35,ICICIGI,2252.93,0.39,5.64,13.78,20.19,25376.0,9829.0,184149.0,-17.20,47.0,X-MC,6.61,68.0,0.39,1.33,21.61,X40,ATH,INSURANCE
50,MASFIN,398.61,-3.06,-5.01,28.48,22.05,26508.0,-4905.0,93075.0,-18.25,49.0,H-SC,6.85,164.0,-0.19,0.67,35.29,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,838.00,4.99,-44.28,212.57,74.15,193766.0,-72451.0,91154.0,-264.07,43.0,H-SC,19.99,149.0,-0.37,0.66,16.84,XR,NTT,CHEMICALS
13,BSOFT,831.70,2.71,-21.28,91.55,50.79,101365.0,-29932.0,110721.0,0.89,75.0,H-SC,5.55,151.0,-0.30,0.80,28.92,XR,ATH,IT
15,CAMS,4762.00,2.45,3.62,20.43,24.79,53319.0,9128.0,260984.0,-1.25,55.0,X-SC,2.16,86.0,0.17,1.88,27.60,X40N,NTT,MISC
29,HAPPSTMNDS,1480.71,2.40,-36.44,190.85,84.85,155459.0,-46709.0,81456.0,-24.49,54.0,H-SC,14.02,147.0,-0.30,0.59,4.54,AR,ATH,IT
40,INDUSINDBK,1800.00,2.26,-36.06,107.84,32.89,53235.0,-27842.0,49365.0,-709.47,65.0,L-MC,6.22,259.0,-0.52,0.36,35.95,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,EASEMYTRIP,26.40,-5.12,-57.06,239.33,45.70,181692.0,-100898.0,75917.0,-21.23,51.0,M-SC,23.55,205.0,-0.56,0.55,9.42,XY24,NTT,TRAVEL
44,JCHAC,2282.00,-3.38,-33.70,50.82,-0.01,43062.0,-43071.0,84734.0,17247.83,8.0,M-SC,0.97,233.0,-1.00,0.61,0.00,OX40N,NTT,AC
50,MASFIN,398.61,-3.06,-5.01,28.48,22.05,26508.0,-4905.0,93075.0,-18.25,49.0,H-SC,6.85,164.0,-0.19,0.67,35.29,XR,ATH,FINANCE
28,GREENPANEL,537.00,-1.98,-36.03,123.75,43.14,144936.0,-65958.0,117120.0,162.27,28.0,M-SC,7.15,240.0,-0.46,0.84,10.17,XY24,NTT,MISC
69,SURYODAY,216.00,-1.67,-21.08,55.90,23.03,78998.0,-37750.0,141321.0,53.51,45.0,H-SC,10.06,167.0,-0.48,1.02,40.22,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.62,-0.52,107.79,106.72,132258.0,-636.0,122700.0,-50.82,38.0,H-SC,1.57,139.0,-0.00,0.88,22.74,AR,ATH,MISC
37,IEX,219.00,-0.40,0.64,47.68,48.63,95946.0,1276.0,201230.0,-33.16,64.0,H-SC,14.73,136.0,0.01,1.45,12.26,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.30,-5.76,120.62,107.91,167827.0,-8507.0,139137.0,-23.39,34.0,M-SC,11.30,216.0,-0.05,1.00,0.30,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.12,-7.65,8.31,0.02,13396.0,-13359.0,161200.0,146.09,68.0,M-SC,15.57,234.0,-1.00,1.16,40.22,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.03,-16.68,20.13,0.09,16902.0,-16813.0,83964.0,94.88,37.0,M-SC,2.75,253.0,-0.99,0.60,10.31,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.28,-23.34,48.02,13.47,99272.0,-62937.0,206730.0,-68.51,33.0,H-SC,2.37,158.0,-0.63,1.49,5.12,XY24,NTT,PAINTS
66,SIS,528.00,0.29,-24.25,60.07,21.26,50526.0,-26920.0,84112.0,1984.46,48.0,H-SC,4.04,166.0,-0.53,0.61,13.64,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.98,5.48,63.31,72.27,113160.0,9292.0,178740.0,-9.26,70.0,M-LC,3.07,99.0,0.08,1.29,12.00,XR,NTT,IT
37,IEX,219.0,-0.40,0.64,47.68,48.63,95946.0,1276.0,201230.0,-33.16,64.0,H-SC,14.73,136.0,0.01,1.45,12.26,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.98,5.48,63.31,72.27,113160.0,9292.0,178740.0,-9.26,70.0,M-LC,3.07,99.0,0.08,1.29,12.00,XR,NTT,IT
37,IEX,219.00,-0.40,0.64,47.68,48.63,95946.0,1276.0,201230.0,-33.16,64.0,H-SC,14.73,136.0,0.01,1.45,12.26,XR,NTT,MISC
38,INDIAMART,4810.62,-0.62,-0.52,107.79,106.72,132258.0,-636.0,122700.0,-50.82,38.0,H-SC,1.57,139.0,-0.00,0.88,22.74,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.01,-1.40,37.55,35.63,77596.0,-2928.0,206646.0,-15.41,42.0,H-MC,3.41,119.0,-0.04,1.49,15.04,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.30,-5.76,120.62,107.91,167827.0,-8507.0,139137.0,-23.39,34.0,M-SC,11.30,216.0,-0.05,1.00,0.30,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.48,-21.07,88.08,48.45,139395.0,-42246.0,158260.0,-26.75,13.0,X-MC,9.41,64.0,-0.30,1.14,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.89,-37.51,116.36,35.20,93560.0,-48264.0,80406.0,5.42,24.0,X-SC,14.40,92.0,-0.52,0.58,0.42,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.42,-11.25,37.00,21.58,46017.0,-15768.0,124371.0,-55.24,26.0,X-MC,5.73,56.0,-0.34,0.90,10.11,X40N,ATH,IT
8,AWL,485.00,-0.38,-21.95,96.67,53.50,227661.0,-66248.0,235503.0,-62.23,27.0,X-MC,1.50,58.0,-0.29,1.70,4.60,XY24,NTT,FMCG
23,DMART,5391.45,0.17,-6.94,37.67,28.12,73764.0,-14592.0,195815.0,-19.69,30.0,X-LC,4.21,19.0,-0.20,1.41,15.05,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-1.11,-23.53,114.65,64.15,208629.0,-55981.0,181970.0,-55.44,42.0,X-SC,1.48,82.0,-0.27,1.31,2.08,XY24,BTT,CEMENT
8,AWL,485.0,-0.38,-21.95,96.67,53.50,227661.0,-66248.0,235503.0,-62.23,27.0,X-MC,1.50,58.0,-0.29,1.70,4.60,XY24,NTT,FMCG
43,ITC,452.0,0.45,-1.71,12.35,10.44,24296.0,-3413.0,196725.0,-41.15,44.0,X-LC,1.58,5.0,-0.14,1.42,3.76,X40,NTT,FMCG
15,CAMS,4762.0,2.45,3.62,20.43,24.79,53319.0,9128.0,260984.0,-1.25,55.0,X-SC,2.16,86.0,0.17,1.88,27.60,X40N,NTT,MISC
36,ICICIPRULI,790.0,0.21,1.55,28.97,30.97,51994.0,2739.0,179477.0,-22.29,49.0,X-MC,2.21,75.0,0.05,1.29,14.31,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.48,-21.07,88.08,48.45,139395.0,-42246.0,158260.0,-26.75,13.0,X-MC,9.41,64.0,-0.30,1.14,0.00,X40N,ATH,FINANCE
55,QUESS,424.00,-1.17,-30.94,106.83,42.84,47961.0,-20111.0,44895.0,-54.82,35.0,X-SC,37.32,83.0,-0.42,0.32,0.00,XY24,NTT,MISC
11,BATAINDIA,2096.00,-0.89,-37.51,116.36,35.20,93560.0,-48264.0,80406.0,5.42,24.0,X-SC,14.40,92.0,-0.52,0.58,0.42,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,0.79,-19.56,75.93,41.51,160850.0,-51525.0,211840.0,-4.10,36.0,X-MC,7.36,61.0,-0.32,1.52,0.79,XY25,ATH,FMCG
54,PGHH,17907.65,0.91,-4.74,40.41,33.76,77308.0,-9510.0,191310.0,-32.78,37.0,X-MC,4.63,57.0,-0.12,1.38,0.91,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.17,-30.94,106.83,42.84,47961.0,-20111.0,44895.0,-54.82,35.0,X-SC,37.32,83.0,-0.42,0.32,0.00,XY24,NTT,MISC
59,RELAXO,1176.00,0.70,-47.03,187.53,52.31,144196.0,-68268.0,76892.0,-43.57,39.0,X-SC,5.33,91.0,-0.47,0.55,2.92,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.89,-37.51,116.36,35.20,93560.0,-48264.0,80406.0,5.42,24.0,X-SC,14.40,92.0,-0.52,0.58,0.42,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.42,-11.25,37.00,21.58,46017.0,-15768.0,124371.0,-55.24,26.0,X-MC,5.73,56.0,-0.34,0.90,10.11,X40N,ATH,IT
51,MEDANTA,1486.00,-1.63,-0.41,24.59,24.08,31088.0,-524.0,126426.0,-8.54,37.0,X-SC,5.94,89.0,-0.02,0.91,19.12,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.74,-12.10,35.69,19.28,108537.0,-41847.0,304109.0,-23.30,70.0,X-LC,6.53,1.0,-0.39,2.19,12.42,X40,ATH,IT
41,INFY,2275.00,0.92,8.46,42.79,54.87,145218.0,26485.0,339373.0,-15.65,67.0,X-LC,2.98,2.0,0.18,2.44,17.45,X40,BTT,IT
75,TMPV,600.00,-0.20,-15.60,68.47,42.18,159725.0,-43132.0,233278.0,-24.30,31.0,X-LC,3.55,3.0,-0.27,1.68,1.05,XY24,NTT,AUTO
81,VBL,671.64,1.24,-2.35,38.94,35.67,120094.0,-7433.0,308409.0,-13.89,61.0,X-LC,5.29,4.0,-0.06,2.22,11.17,X40N,ATH,FMCG
43,ITC,452.00,0.45,-1.71,12.35,10.44,24296.0,-3413.0,196725.0,-41.15,44.0,X-LC,1.58,5.0,-0.14,1.42,3.76,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,2.26,-36.06,107.84,32.89,53235.0,-27842.0,49365.0,-709.47,65.0,L-MC,6.22,259.0,-0.52,0.36,35.95,XR,NTT,BANKS
6,ASIANTILES,137.00,0.19,-14.92,113.53,81.67,90614.0,-13995.0,79815.0,7172.22,45.0,L-SC,16.42,271.0,-0.15,0.57,54.57,XR,NTT,CERAMICS
50,MASFIN,398.61,-3.06,-5.01,28.48,22.05,26508.0,-4905.0,93075.0,-18.25,49.0,H-SC,6.85,164.0,-0.19,0.67,35.29,XR,ATH,FINANCE
13,BSOFT,831.70,2.71,-21.28,91.55,50.79,101365.0,-29932.0,110721.0,0.89,75.0,H-SC,5.55,151.0,-0.30,0.80,28.92,XR,ATH,IT
69,SURYODAY,216.00,-1.67,-21.08,55.90,23.03,78998.0,-37750.0,141321.0,53.51,45.0,H-SC,10.06,167.0,-0.48,1.02,40.22,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.12,-7.65,8.31,0.02,13396.0,-13359.0,161200.0,146.09,68.0,M-SC,15.57,234.0,-1.00,1.16,40.22,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.15,3.08,17.33,20.95,44977.0,7762.0,259530.0,2.04,78.0,X-LC,17.92,31.0,0.17,1.87,39.82,X40,ATH,PAINTS
13,BSOFT,831.70,2.71,-21.28,91.55,50.79,101365.0,-29932.0,110721.0,0.89,75.0,H-SC,5.55,151.0,-0.30,0.80,28.92,XR,ATH,IT
40,INDUSINDBK,1800.00,2.26,-36.06,107.84,32.89,53235.0,-27842.0,49365.0,-709.47,65.0,L-MC,6.22,259.0,-0.52,0.36,35.95,XR,NTT,BANKS
32,HCLTECH,1922.01,0.31,6.46,17.16,24.73,44197.0,15634.0,257558.0,11.06,64.0,X-LC,5.09,7.0,0.35,1.85,22.68,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.38
1,25,45.29
2,50,76.24


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.17
MC    29.55
LC    24.30
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.95
X40      21.91
X40N     12.19
XR        9.95
AR        9.26
XY25      9.22
OX40N     7.72
SR        1.02
MH        0.80
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.35
X-MC    22.94
X-LC    20.81
M-SC    12.05
X-SC     8.37
H-MC     4.85
M-MC     1.40
L-SC     1.40
M-LC     1.29
H-LC     1.20
L-LC     1.00
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.10,-6.04,40.33
IT,12.52,-15.88,78.53
FINANCE,9.46,-18.03,67.90
MISC,7.60,-22.90,73.25
ELECTRICAL,6.15,-9.74,49.94
PAINTS,6.15,-8.19,24.26
INSURANCE,4.44,-2.52,38.64
PHARMA,3.99,-0.83,33.10
AUTO,3.41,-21.35,73.38


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3153698.0,21
AR,1283912.0,10
XR,1281295.0,13
X40,996117.0,14
X40N,756635.0,9
OX40N,720297.0,10
XY25,359772.0,6
SR,277928.0,2
MH,72394.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3595813.0,25
M-SC,1383237.0,15
X-MC,1298803.0,16
X-LC,872222.0,11
X-SC,757720.0,8
H-MC,404336.0,3
L-SC,265461.0,3
M-LC,113160.0,1
H-LC,66749.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1260248.0      6
           AR         883220.0      5
M-SC       XY24       792026.0      6
H-SC       XR         768696.0      7
X-MC       X40        449989.0      7
           XY24       386860.0      3
X-LC       X40        367225.0      5
H-SC       OX40N      333327.0      4
M-SC       OX40N      299870.0      5
X-SC       X40N       291139.0      3
           XY24       287678.0      3
H-SC       SR         277928.0      2
X-MC       X40N       271638.0      4
X-LC       XY24       232995.0      2
H-MC       AR         210445.0      2
           XY24       193891.0      1
X-LC       X40N       193858.0      2
X-MC       XY25       190316.0      2
X-SC       X40        178903.0      2
L-SC       XR         178361.0      2
M-SC       XR         167843.0      2
           AR         123498.0      2
M-LC       XR         113160.0      1
L-SC       OX40N       87100.0      1
X-LC       XY25        78144.0      2
H-SC       MH          72394.0      1
H-LC       AR          66749.0      1
L-MC       XR          53235.0      1
M-MC       XY25        51265.0      1
L-LC       XY25        40047.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
